Comenzamos con el ETL. Tenemos el dataset seleccionado, importamos libreria pandas para poder leerlo. 

In [39]:
import pandas as pd 

Importamos el csv y le damos una primer vista previa. 

In [40]:
data = pd.read_csv('ADNI_Training_Q3_APOE_CollectionADNI1Complete 1Yr 1.5T_July22.2014.csv')

In [41]:
data.head()

,directory.id,Subject,RID,Image.Data.ID,Modality,Visit,Acq.Date,DX.bl,EXAMDATE,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,APOE4,MMSE,imputed_genotype,APOE Genotype,Dx Codes for Submission
0,4702245ea294ce5d4e9b8a87027dfdf4,011_S_0003,3,32237,MRI,1,09-01-2005,AD,09-12-2005,81.3,Male,18,Not Hisp/Latino,White,1,20,True,"3,4",AD
1,2e89e352af743597b2368c412e0f6de2,022_S_0004,4,64631,MRI,1,9/22/05,LMCI,11-08-2005,67.5,Male,10,Hisp/Latino,White,0,27,False,"3,3",MCI
2,90419199306997753de8042f1fd55e38,011_S_0005,5,32246,MRI,1,09-02-2005,CN,09-07-2005,73.7,Male,16,Not Hisp/Latino,White,0,29,True,"3,3",CN
3,d8d175ffff1e2053e6a18c5df494ccdf,100_S_0006,6,33025,MRI,1,11/15/05,LMCI,11/29/05,80.4,Female,13,Not Hisp/Latino,White,0,25,True,"3,3",MCI
4,986e75b2e604cd44b38feb2188476fb2,011_S_0010,10,32270,MRI,1,11-07-2005,AD,11-10-2005,73.9,Female,12,Not Hisp/Latino,White,1,24,True,"3,4",AD


In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 628 entries, 0 to 627
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   directory.id             628 non-null    object 
 1   Subject                  628 non-null    object 
 2   RID                      628 non-null    int64  
 3   Image.Data.ID            628 non-null    int64  
 4   Modality                 628 non-null    object 
 5   Visit                    628 non-null    int64  
 6   Acq.Date                 628 non-null    object 
 7   DX.bl                    628 non-null    object 
 8   EXAMDATE                 628 non-null    object 
 9   AGE                      628 non-null    float64
 10  PTGENDER                 628 non-null    object 
 11  PTEDUCAT                 628 non-null    int64  
 12  PTETHCAT                 628 non-null    object 
 13  PTRACCAT                 628 non-null    object 
 14  APOE4                    6

Vamos a comenzar borrando aquellas columnas que no sirven para el proyecto, por ejemplo las relacionadas a ID o valores unicos. 

In [43]:
eliminar = ['directory.id', 'Subject', 'RID', 'Image.Data.ID', 'Modality', 'Visit', 'Acq.Date', 'EXAMDATE']

data = data.drop(eliminar, axis=1)

In [44]:
data.head()

,DX.bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,APOE4,MMSE,imputed_genotype,APOE Genotype,Dx Codes for Submission
0,AD,81.3,Male,18,Not Hisp/Latino,White,1,20,True,"3,4",AD
1,LMCI,67.5,Male,10,Hisp/Latino,White,0,27,False,"3,3",MCI
2,CN,73.7,Male,16,Not Hisp/Latino,White,0,29,True,"3,3",CN
3,LMCI,80.4,Female,13,Not Hisp/Latino,White,0,25,True,"3,3",MCI
4,AD,73.9,Female,12,Not Hisp/Latino,White,1,24,True,"3,4",AD


Vamos a renombrar columna DX.bi para evitar inconvenientes futuros, y la vamos a explorar un poco más ya que parecería nuestro target. 

In [45]:
data = data.rename(columns={'DX.bl': 'DX', 'Dx Codes for Submission': 'DX_SUB'})

Se recurre a una búsqueda bibliografica para determinar el significado de los valores de DX: LMCI (deterioro cognitivo leve), CN (cognitivamente normal) y AD (diagnostico de alzheimer)

In [46]:
data.DX.value_counts()

DX
LMCI    305
CN      190
AD      133
Name: count, dtype: int64

In [47]:
data.DX_SUB.value_counts()

DX_SUB
MCI    305
CN     190
AD     133
Name: count, dtype: int64

Vamos a eliminar la columna DX_SUB porque es igual a nuestra columna target "DX"

In [48]:
data.drop('DX_SUB', axis=1)

,DX,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,APOE4,MMSE,imputed_genotype,APOE Genotype
0,AD,81.3,Male,18,Not Hisp/Latino,White,1,20,True,"3,4"
1,LMCI,67.5,Male,10,Hisp/Latino,White,0,27,False,"3,3"
2,CN,73.7,Male,16,Not Hisp/Latino,White,0,29,True,"3,3"
3,LMCI,80.4,Female,13,Not Hisp/Latino,White,0,25,True,"3,3"
4,AD,73.9,Female,12,Not Hisp/Latino,White,1,24,True,"3,4"
...,...,...,...,...,...,...,...,...,...,...
623,LMCI,74.4,Female,12,Not Hisp/Latino,White,1,29,True,"3,4"
624,LMCI,69.4,Male,19,Not Hisp/Latino,White,1,29,True,"3,4"
625,LMCI,75.6,Female,16,Not Hisp/Latino,White,0,28,True,"3,3"
626,LMCI,83.4,Male,18,Not Hisp/Latino,White,0,26,True,"3,3"


Vamos a guardar este dataset para realizar el EDA 

In [49]:
data.to_csv('data_EDA.csv')

Vamos a codificar las variables nominales. En su mayoria son categoricas, por lo tanto vamos a aplicar el OneHotEncoder

In [50]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

columnas_categoricas = ['DX', 'PTGENDER', 'PTETHCAT', 'PTRACCAT', 'imputed_genotype']

for x in columnas_categoricas:

    data[x] = data[x].astype('category')

    type_encode = LabelEncoder()

    data[x] = type_encode.fit_transform(data[x])

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 628 entries, 0 to 627
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DX                628 non-null    int32  
 1   AGE               628 non-null    float64
 2   PTGENDER          628 non-null    int32  
 3   PTEDUCAT          628 non-null    int64  
 4   PTETHCAT          628 non-null    int32  
 5   PTRACCAT          628 non-null    int32  
 6   APOE4             628 non-null    int64  
 7   MMSE              628 non-null    int64  
 8   imputed_genotype  628 non-null    int64  
 9   APOE Genotype     628 non-null    object 
 10  DX_SUB            628 non-null    object 
dtypes: float64(1), int32(4), int64(4), object(2)
memory usage: 44.3+ KB


In [52]:
data.head()

,DX,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,APOE4,MMSE,imputed_genotype,APOE Genotype,DX_SUB
0,0,81.3,1,18,1,2,1,20,1,"3,4",AD
1,2,67.5,1,10,0,2,0,27,0,"3,3",MCI
2,1,73.7,1,16,1,2,0,29,1,"3,3",CN
3,2,80.4,0,13,1,2,0,25,1,"3,3",MCI
4,0,73.9,0,12,1,2,1,24,1,"3,4",AD


In [53]:
columnas_cat = ['DX', 'PTGENDER', 'PTETHCAT', 'PTRACCAT']

type_one_hot = OneHotEncoder()

for x in columnas_cat: 
    type_one_hot_encode = type_one_hot.fit_transform(data[x].values.reshape(-1,1)).toarray()
    ohe_variable = pd.DataFrame(type_one_hot_encode, columns = [x+str(int(i)) for i in range(type_one_hot_encode.shape[1])])
    data = pd.concat([data, ohe_variable], axis=1)
    data = data.drop(x, axis = 1)

data.head()

,AGE,PTEDUCAT,APOE4,MMSE,imputed_genotype,APOE Genotype,DX_SUB,DX0,DX1,DX2,PTGENDER0,PTGENDER1,PTETHCAT0,PTETHCAT1,PTETHCAT2,PTRACCAT0,PTRACCAT1,PTRACCAT2
0,81.3,18,1,20,1,"3,4",AD,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,67.5,10,0,27,0,"3,3",MCI,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
2,73.7,16,0,29,1,"3,3",CN,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,80.4,13,0,25,1,"3,3",MCI,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,73.9,12,1,24,1,"3,4",AD,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


Guaramos lo obtenido en un csv listo para utilizar para entrenamiento de modelo de machinelearning.

In [54]:
data.to_csv('data_ML.csv')